In [1]:
#import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import glob
import matplotlib

In [2]:
deeplabv3plus_dir="./src"
sys.path.append(deeplabv3plus_dir)

In [3]:
gpu_options = tf.compat.v1.GPUOptions(visible_device_list="1", allow_growth=True)
config = tf.compat.v1.ConfigProto(gpu_options = gpu_options)
tf.compat.v1.enable_eager_execution(config=config)

In [4]:
#from model import deeplab_v3plus
from data_utils import make_xy_from_data_paths, convert_y_to_image_array
from data_gen import DataGenerator
from label import Label

In [5]:
from metrics import IoU
from tensorflow.keras.utils import get_custom_objects
get_custom_objects()["IoU"] = IoU

In [6]:
#import importlib
#importlib.reload(sys.modules['data_utils'])

In [7]:
model_dir = "../rot_rgb"
out_dir = os.path.join(model_dir,"figure")
model = keras.models.load_model(os.path.join(model_dir,'best_model.h5'))

W1224 15:51:48.462549 140735459642576 deprecation.py:323] From /home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
preprocess = keras.applications.xception.preprocess_input

In [9]:
#model.summary()

In [10]:
traindata_dir = '../train_data'
#testdata_dir = '../kaigan_block/dataset/x_test2'

train_x_paths = glob.glob(os.path.join(traindata_dir,'*.jpg'))
train_x_paths.sort()
image_names = [os.path.basename(train_x_paths[i]).split('.')[0] for i in range(len(train_x_paths))]
train_y_paths=[]
for i, image_name in enumerate(image_names):
    p = os.path.join(traindata_dir, image_name+'.json')
    if os.path.exists(p):
        train_y_paths.append(p)
    else:
        train_y_paths.append(None)

In [11]:
label_file_path = os.path.join(traindata_dir, 'label_list.csv')
label = Label(label_file_path)
image_size = (512,512)

# inference for trained_data

In [12]:
valid_x, valid_y = make_xy_from_data_paths(train_x_paths,
                                           train_y_paths,
                                           image_size,
                                           label,
                                           "polygon",
                                           resize_or_crop="crop")

In [13]:
#valid_x = make_x_from_image_paths(img_paths[0:50], image_size)
#valid_y = make_y_from_image_paths(seg_img_paths[0:50], image_size, n_categories)

In [14]:
pred = model.predict(preprocess(valid_x))

In [15]:
y_pred = convert_y_to_image_array(pred, label, threshold=0.5)
y_true = convert_y_to_image_array(valid_y, label)

In [16]:

out_dir_train = os.path.join(out_dir, "train")
os.makedirs(out_dir_train,exist_ok = True)
matplotlib.use('Agg')
for i in range(len(y_pred)):
    plt.figure(figsize=(20,30))
    plt.subplot(3,2,1)
    plt.imshow(y_pred[i])
    plt.title("pred", fontsize=20)
    plt.grid()
    plt.subplot(3,2,2)
    plt.title("true", fontsize=20)
    plt.imshow(y_true[i])
    plt.grid()
    plt.subplot(3,2,3)
    plt.imshow(valid_x[i,:,:,:])
    plt.grid()
    plt.subplot(3,2,4)
    plt.imshow(valid_x[i,:,:,:])
    plt.grid()

    y_mask = y_pred[i].copy()/255
    black_pix=(y_mask == np.array([0,0,0])).all(axis=2)
    y_mask[black_pix,:] = [1.0,1.0,1.0]
    plt.subplot(3,2,5)
    plt.imshow(y_mask*valid_x[i,:,:,:]/255)
    plt.grid()
    y_mask = y_true[i].copy()/255
    black_pix=(y_mask == np.array([0,0,0])).all(axis=2)
    y_mask[black_pix,:] = [1.0,1.0,1.0]
    plt.subplot(3,2,6)
    plt.imshow(y_mask*(valid_x[i,:,:,:]/255))
    plt.grid()

    plt.savefig(os.path.join(out_dir_train,str(i).zfill(6) + ".jpg"))
    plt.close()

# inference for validation data

In [17]:
valid_data_dir = '../valid_data/select'
#testdata_dir = '../kaigan_block/dataset/x_test2'

valid_x_paths = glob.glob(os.path.join(valid_data_dir,'*.jpg'))
valid_x_paths.sort()
len(valid_x_paths)

16

In [18]:
tar = range(len(valid_x_paths))

In [19]:
from data_utils import inference_large_img
from PIL import Image
from tqdm import tqdm

In [20]:
mode = "simple_crop"
out_dir_valid = os.path.join(out_dir, "valid", mode)
os.makedirs(out_dir_valid, exist_ok=True)
matplotlib.use('Agg')
for i in tqdm(tar):
    x_img, seg_img = inference_large_img(valid_x_paths[i],
                                         model,
                                         preprocess,
                                         label,
                                         mode=mode,
                                         threshold=0.5)

    figsize = np.array(x_img.shape[0:2][::-1])/100

    plt.figure(figsize=figsize)
    plt.imshow(seg_img)
    #plt.grid()
    plt.axis('off')
    plt.savefig(os.path.join(out_dir_valid, str(i).zfill(6) + "_seg.jpg"), bbox_inches='tight')
    plt.close()

    plt.figure(figsize=figsize)
    plt.imshow(x_img)
    #plt.grid()
    plt.axis('off')
    plt.savefig(os.path.join(out_dir_valid, str(i).zfill(6) + "_x.jpg"), bbox_inches='tight')
    plt.close()

    plt.figure(figsize=figsize)
    y_mask = seg_img.copy()/255
    black_pix=(y_mask == np.array([0,0,0])).all(axis=2)
    y_mask[black_pix,:] = [1.0,1.0,1.0]
    plt.imshow(y_mask*x_img/255)
    #plt.grid()
    plt.axis('off')
    plt.savefig(os.path.join(out_dir_valid, str(i).zfill(6) + "_x_seg.jpg"), bbox_inches='tight')
    plt.close()

  0%|          | 0/16 [00:00<?, ?it/s]

predicting
converting y to PIL image
merging image


  6%|▋         | 1/16 [00:27<06:51, 27.44s/it]

predicting
converting y to PIL image
merging image


 12%|█▎        | 2/16 [00:53<06:17, 26.95s/it]

...

 88%|████████▊ | 14/16 [06:01<00:51, 25.65s/it]

predicting
converting y to PIL image
merging image


 94%|█████████▍| 15/16 [06:27<00:25, 25.68s/it]

predicting
converting y to PIL image
merging image


100%|██████████| 16/16 [06:53<00:00, 25.68s/it]


In [21]:
mode = "center"
out_dir_valid = os.path.join(out_dir, "valid", mode)
os.makedirs(out_dir_valid, exist_ok=True)
matplotlib.use('Agg')
for i in tqdm(tar):
    x_img, seg_img = inference_large_img(valid_x_paths[i],
                                         model,
                                         preprocess,
                                         label,
                                         mode=mode,
                                         threshold=0.5)

    figsize = np.array(x_img.shape[0:2][::-1])/100

    plt.figure(figsize=figsize)
    plt.imshow(seg_img)
    #plt.grid()
    plt.axis('off')
    plt.savefig(os.path.join(out_dir_valid, str(i).zfill(6) + "_seg.jpg"), bbox_inches='tight')
    plt.close()

    plt.figure(figsize=figsize)
    plt.imshow(x_img)
    #plt.grid()
    plt.axis('off')
    plt.savefig(os.path.join(out_dir_valid, str(i).zfill(6) + "_x.jpg"), bbox_inches='tight')
    plt.close()

    plt.figure(figsize=figsize)
    y_mask = seg_img.copy()/255
    black_pix=(y_mask == np.array([0,0,0])).all(axis=2)
    y_mask[black_pix,:] = [1.0,1.0,1.0]
    plt.imshow(y_mask*x_img/255)
    #plt.grid()
    plt.axis('off')
    plt.savefig(os.path.join(out_dir_valid, str(i).zfill(6) + "_x_seg.jpg"), bbox_inches='tight')
    plt.close()

  0%|          | 0/16 [00:00<?, ?it/s]

predicting
converting y to PIL image
merging image


  6%|▋         | 1/16 [00:40<10:14, 40.96s/it]

predicting
converting y to PIL image
merging image


 12%|█▎        | 2/16 [01:20<09:28, 40.60s/it]

...

 88%|████████▊ | 14/16 [09:15<01:19, 39.54s/it]

predicting
converting y to PIL image
merging image


 94%|█████████▍| 15/16 [09:55<00:39, 39.65s/it]

predicting
converting y to PIL image
merging image


100%|██████████| 16/16 [10:35<00:00, 39.68s/it]


In [22]:
mode = "max_confidence"
out_dir_valid = os.path.join(out_dir, "valid", mode)
os.makedirs(out_dir_valid, exist_ok=True)
matplotlib.use('Agg')
for i in tqdm(tar):
    x_img, seg_img = inference_large_img(valid_x_paths[i],
                                         model,
                                         preprocess,
                                         label,
                                         mode=mode,
                                         threshold=0.5)

    figsize = np.array(x_img.shape[0:2][::-1])/100

    plt.figure(figsize=figsize)
    plt.imshow(seg_img)
    #plt.grid()
    plt.axis('off')
    plt.savefig(os.path.join(out_dir_valid, str(i).zfill(6) + "_seg.jpg"), bbox_inches='tight')
    plt.close()

    plt.figure(figsize=figsize)
    plt.imshow(x_img)
    #plt.grid()
    plt.axis('off')
    plt.savefig(os.path.join(out_dir_valid, str(i).zfill(6) + "_x.jpg"), bbox_inches='tight')
    plt.close()

    plt.figure(figsize=figsize)
    y_mask = seg_img.copy()/255
    black_pix=(y_mask == np.array([0,0,0])).all(axis=2)
    y_mask[black_pix,:] = [1.0,1.0,1.0]
    plt.imshow(y_mask*x_img/255)
    #plt.grid()
    plt.axis('off')
    plt.savefig(os.path.join(out_dir_valid, str(i).zfill(6) + "_x_seg.jpg"), bbox_inches='tight')
    plt.close()

  0%|          | 0/16 [00:00<?, ?it/s]

predicting
merging image
converting y to PIL image


  6%|▋         | 1/16 [01:38<24:44, 98.97s/it]

predicting
merging image
converting y to PIL image


 12%|█▎        | 2/16 [03:18<23:08, 99.19s/it]

...

 88%|████████▊ | 14/16 [23:15<03:20, 100.06s/it]

predicting
merging image
converting y to PIL image


 94%|█████████▍| 15/16 [24:56<01:40, 100.31s/it]

predicting
merging image
converting y to PIL image


100%|██████████| 16/16 [26:37<00:00, 100.38s/it]
